## Представление триангуляции в памяти

Для хранения триангуляции заведём структуры для хранения вершин и фейсов. Хранить рёбра мы не будем

In [1]:
class TVertex:
    """
    Класс точки.
    Хранит координаты, и один из фейсов, точкой которого она является
    """
    def __init__(self, x, y, face):
        self.x = x
        self.y = y
        self.face = face
        
class TFace:
    # todo: узнать по часовой или против
    """
    Класс фейса.
    Хранит список вершин, отсортированный в порядке обхода против часовой стрелки и список соседних фейсов
    такой, что neighbours[i] — фейс, находящийся напротив вершины vertices[i]
    """
    def __init__(self, vertices, neighbours):
        self.vertices = vertices
        self.neighbours = neighbours

### Оценка эффективности по памяти нашей структуры по сравнению с обычным DCEL

Определим обычный DCEL

In [2]:
class Vertex:
    """
    Класс точки.
    Хранит полярные координаты, и какое-нибудь полуребро, имеющее начало в этой точке (edge.start == this)
    """
    def __init__(self, x, y, edge):
        self.x = x
        self.y = y
        self.edge = edge
        
class Face:
    """
    Класс фейса.
    Хранит какое-нибудь ребро, находящееся на границе этого фейса (edge.incident_face == this)
    """
    def __init__(self, edge):
        self.edge = edge
        
class HalfEdge:
    """
    Класс полуребра. Для каждого ребра (неориентированного) исходного графа из вершины U в V создаём
    2 полуребра (ориентированных): U -> V и V -> U
    Объект хранит:
    - указатели на точку, откуда исходит ребро (start),
    - указатель на ребро-близнец (twin) (направленное в другую сторону),
    - инцидентную поверхность (incident_face),
    - указатели на следующее и предыдущие ребра.
    """
    def __init__(self, prev, next, twin, start, incident_face):
        self.prev = prev # prev.next == this
        self.next = next # next.prev == this
        self.twin = twin # twin.twin == this
        self.start = prev # twin.next.origin == origin and prev.twin.origin == origin
        self.incident_face = incident_face # prev.incident_face == incident_face and next.incident_face == incident_face

Размер структуры TVertex совпадает с размером структуры Vertex. В отличие от обычного DCEL мы не храним рёбра, что экономит нам 6 $\cdot$ 2 = 12 ссылок для каждого ребра. С другой стороны, в каждом фейсе мы храним 3 ссылки на вершины фейса и три ссылки на смежные фейсы, что увеличивает размер структуры TFace относительно Face на 5 ссылок, соответственно общий размер триангуляции увеличивается на 5 ссылок для каждого фейса. Количество полурёбер в триангулции находится по формуле E = F $\cdot$ 3 + V (E - полурёбра, V - вершины, F - фейсы). Дуопустим в триангуляции у нас V вершин и F фейсов. Тогда, сохранив триангуялцию в нашу структуру мы сэкономим (F $\cdot$ 3 + V) $\cdot$ 6 ссылок, но дополнительно потратим 5 $\cdot$ F ссылок. В итоге получается 18F + 6V - 5F = 13F + 6V сэкономленных ссылок

## Некоторые полезные операции с нашей структурой

### Обход фейсов вокруг точки

#### Алгоритм

У нас есть точка, неоходимо обойти все фейсы вокруг неё против часовой стрелки. Из точки можно попасть в какой-то смежный ей фейс. Затем, переберём список вершин этого фейса, найдём на какой позиции в этом списке находится наша точка. Допустим, это позиция i. Тогда neighbours[i] — фейс, находящийся напротив нашей точки. Затем достаточно взять следующего соседа (neighbours[i + 1 по модулю 3]), это и будет следующим фейсом в обходе против часовой стрелки. Повторять алгоритм пока не встретим фейс, с которого мы начали обход.

#### Корректность

В какой-то смежный фейс из точки можно попасть, потому что точка хранит ссылку на него. Фейс, находящийся напротив нашей точки находим благодаря тому, что для любого i neighbours[i] — фейс, находящийся напротив вершины vertices[i], а раз фейс смежный нашей точке, то существует i такое, что vertices[i] — наша точка. Рассмотрим фейс, смежный какой-то точке. У него есть 2 ребра, инцидентные этой точке и одно ребро, не инцидентное ей. Соседи этого фейса по инцидентным точке рёбрам будут смежны этой точке (вроде очевидно), а оставшийся сосед (сосед по ребру напротив точки) не будет смежен этой вершине, так как если бы он был ей смежен, то все 3 его точки совпали бы с точками фейса, который мы рассматриваем, т.е. сосед фейса совпал бы с этим фейсом, что не может произойти. Таким образом, у смежного точке фейса 2 соседа будут смежны этой точке, а сосед напротив точки не будет ей смежен. Если мы возьмём соседний фейс напротив нашей точки, возьмём следующего соседа в порядке обхода против часовой стрелки, то мы получим фейс, смежный с нашей точкой, причём являющийся следующим в обходе против часовой стрелки.

#### Время работы

Допустим существет N смежных с точкой фейсов.
Получаем смежный с точкой фейс 1 раз за O(1) Затем для каждого фейса, смежного с точкой необходимо найти позицию точки в списке vertices, его длина 3, значит это делается за O(1). Далее за константу находим следующий фейс.
Итого O(1) + N $\cdot$ O(1) = O(N)

### Перебор фейсов в порядке обхода луча

//todo
[ответ в том, чтобы создать итератор
который
1) хранит в себе, что он последнее вернул - треугольник или точку (т.к. луч может пересечь точку)
2) если точку, то перебирает все инцидентные треугольники по часовой [как перебирать ты тоже должен рассказать, отдельно]
3) если треугольник, за единицу времени определяет отрезок треугольника, который этот луч пересекает, и по отрезку возвращает треугольник, инцидентный отрезку
третий пункт сложно мб осознать, но на самом деле изи, порисуй]

